# Fairy Tale with RNN


Feedforward networks feed information straight straight through (never touching a given node twice), while recurrent neural networks cycle it through a loop.

In the case of feedforward networks, input examples are fed to the network and transformed into an output; with supervised learning, the output would be a label, a name applied to the input.
A feedforward network is trained on labeled inputs until it minimizes the error it makes when guessing their categories. A trained feedforward network can be exposed to any random collection of inputs, and the first input it is exposed to will not necessarily alter how it classifies the second. That is, a feedforward network has no notion of order in time, and the only input it considers is the current example it has been exposed to. Feedforward networks are amnesiacs regarding their recent past; they remember nostalgically only the formative moments of training.

Recurrent networks, on the other hand, take as their input not just the current input example they see, but also what they have perceived previously in time. The decision a recurrent net reached at time step t-1 affects the decision it will reach one moment later at time step t. So recurrent networks have two sources of input, the present and the recent past, which combine to determine how they respond to new data. Recurrent networks are distinguished from feedforward networks by that feedback loop connected to their past decisions, ingesting their own outputs moment after moment as input. Recurrent networks have memory. Adding memory to neural networks has a purpose: there is information in the sequence itself, and recurrent nets use it to perform tasks that feedforward networks can’t. 
That sequential information is preserved in the recurrent network’s hidden state, which manages to span many time steps as it cascades forward to affect the processing of each new example. It is finding correlations between events separated by many moments, and these correlations are called “long-term dependencies”, because an event downstream in time depends upon, and is a function of, one or more events that came before. One way to think about RNNs is this: they are a way to share weights over time.
We can describe the process of carrying memory forward mathematically:

h_t=/phi(Wx_t+Uh_{t-1})

The hidden state at time step t is h_t. It is a function of the input at the same time step x_t, modified by a weight matrix W (like the one we used for feedforward nets) added to the hidden state of the previous time step h_{t-1} multiplied by its own hidden-state-to-hidden-state matrix U, otherwise known as a transition matrix. The weight matrices are filters that determine how much importance to accord to both the present input and the past hidden state. The error they generate will return via backpropagation and be used to adjust their weights until error can’t go any lower.
The sum of the weight input and hidden state is squashed by the function φ, either a logistic sigmoid function or tanh.
Because this feedback loop occurs at every time step in the series, each hidden state contains traces not only of the previous hidden state, but also of all those that preceded h_{t-1} for as long as memory can persist.
Given a series of letters, a recurrent neural network will use the first character to help determine its perception of the second character, such that an initial q might lead it to infer that the next letter will be u, while an initial t might lead it to infer that the next letter will be h.


In [4]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn 
import os
import time
import math
import numpy as np
import my_txtutils as txt
tf.set_random_seed(0)

# Training with H. C. Andersen

In [5]:
SEQLEN = 30
BATCHSIZE = 200
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512 
NLAYERS = 3
learning_rate = 0.001  
dropout_pkeep = 0.8    

# load data

andersendir = "andersen/*.txt"
codetext, valitext, bookranges = txt.read_data_files(andersendir, validation=True) 

# txt.read_data_files reads data files, optionnaly set aside the last file as validation data, returns: 
# training data, validation data, list of loaded file names with ranges
     
# display some stats on the data

epoch_size = len(codetext) // (BATCHSIZE * SEQLEN) 
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

lr = tf.placeholder(tf.float32, name='lr')  # learning rate
pkeep = tf.placeholder(tf.float32, name='pkeep')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')

# inputs

X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# expected outputs = same sequence shifted by 1 since we are trying to predict the next character

Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# input state

Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using a NLAYERS=3 layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

# How to properly apply dropout in RNNs: see README.md
# (optimizing any neural network with dropout is equivalent to a form of approximate Bayesian inference.) 

cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

# "naive dropout" implementation

dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)

# dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)

# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence

H = tf.identity(H, name='H')  # just to give it a name

# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
# From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# stats for display

seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
loss_summary = tf.summary.scalar("batch_loss", batchloss)
acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

# Init Tensorboard stuff. This will save Tensorboard information into a different
# folder at each run named 'log/<timestamp>/'. Two sets of data are saved so that
# you can compare training and validation curves visually in Tensorboard.

timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter("log/" + timestamp + "-training")
validation_writer = tf.summary.FileWriter("log/" + timestamp + "-validation")

# Init for saving models. They will be saved into a directory named 'checkpoints'.
# Only the last checkpoint is kept.

if not os.path.exists("checkpoints"):
    os.mkdir("checkpoints")
saver = tf.train.Saver(max_to_keep=1000)

# for display: init the progress bar

DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init

istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state

#init = tf.global_variables_initializer() 
# comment the previous out when you want to restore the model

sess = tf.Session()

#sess.run(init)

step = 0

saver.restore(sess, "/Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_1519939554-0") 
# edit with the right name

# training loop

for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):

    # train on one minibatch
    
    feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
    
    if step % _50_BATCHES == 0:
        feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
        y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
        summary_writer.add_summary(smm, step)

    # run a validation step every 50 batches
    # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
    # so we cut it up and batch the pieces (slightly inaccurate)
    # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.
    
    if step % _50_BATCHES == 0 and len(valitext) > 0:
        VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
        bsize = len(valitext) // VALI_SEQLEN
        txt.print_validation_header(len(codetext), bookranges)
        vali_x, vali_y, _ = next(txt.rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
        vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
        feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                     batchsize: bsize}
        ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_validation_stats(ls, acc)
        # save validation data for Tensorboard
        validation_writer.add_summary(smm, step)

    # display a short text generated with the current weights and biases (every 150 batches)
   
    if step // 3 % _50_BATCHES == 0:
        txt.print_text_generation_header()
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])
        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            print(chr(txt.convert_to_alphabet(rc)), end="")
            ry = np.array([[rc]])
        txt.print_text_generation_footer()

    # save a checkpoint (every 500 batches)
    
    if step // 10 % _50_BATCHES == 0:
        saved_file = saver.save(sess, "/Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_" + timestamp, global_step=step)
        print("Saved file: " + saved_file)

    # display progress bar
    
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    
    istate = ostate
    step += BATCHSIZE * SEQLEN

Loading file andersen/andersen_part25.txt
Loading file andersen/andersen_part19.txt
Loading file andersen/andersen_part18.txt
Loading file andersen/andersen_part24.txt
Loading file andersen/andersen_part26.txt
Loading file andersen/andersen_part27.txt
Loading file andersen/andersen_part23.txt
Loading file andersen/andersen_part22.txt
Loading file andersen/andersen_part08.txt
Loading file andersen/andersen_part20.txt
Loading file andersen/andersen_part21.txt
Loading file andersen/andersen_part09.txt
Loading file andersen/andersen_part10.txt
Loading file andersen/andersen_part04.txt
Loading file andersen/andersen_part05.txt
Loading file andersen/andersen_part11.txt
Loading file andersen/andersen_part07.txt
Loading file andersen/andersen_part13.txt
Loading file andersen/andersen_part12.txt
Loading file andersen/andersen_part06.txt
Loading file andersen/andersen_part02.txt
Loading file andersen/andersen_part16.txt
Loading file andersen/andersen_part17.txt
Loading file andersen/andersen_par

VALIDATION STATS:                                  loss: 1.27149,       accuracy: 0.61346

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
jude. Then she called the other shoter bothes; then the wide country lad arove out of the rock where the sunfeamed the meany, and he was always a long latter of heavyed, and he should be so souble it was a short as the lightning to see of its songs, and penhaps fall from the sun through the rovel, and then are above them.

"I who has a beautieul portulath," said the old somemorrigatation to the one of the one man were still splings to her; of, he knew it them so looked like a shape of the window, and the boy at length the tree of the farmshouse were so soon with secing that they are beautiful when she choked, as if he had been a great daughters of a marshe and a bird at the fear him see again.

"That's a few tabt  with his mother and heavy bad shoes, and then she happened for the door

VALIDATION STATS:                                  loss: 1.26687,       accuracy: 0.61481

0%                                        Training on next 50 batches                                        100%
600000 (epoch 0) andersen_part22.txt │ he snuff-box was a toy puzzle. │ e wtodf box aas t grn catzled  │ loss: 2.07345
600030 (epoch 0) andersen_part22.txt │ hest with as many pieces as th │ urt ohth t  sany peeces on the │ loss: 1.13193
600060 (epoch 0) andersen_part22.txt │ ing, and saw the soldiers marc │ ng  and thi the studier  wadeh │ loss: 1.05988
600090 (epoch 0) andersen_part22.txt │  must step to get higher up? O │ wast beap to tot iimher ap  In │ loss: 1.43144
600120 (epoch 0) andersen_part22.txt │ ," said the ball; "but I canno │ " said the morl, "aut I wan ot │ loss: 0.55677
600150 (epoch 0) andersen_part22.txt │ rchyard were several graves, a │ ch ard ware toeeral srene,  an │ loss: 0.98204
600180 (epoch 0) andersen_part22.txt │ owman went at once to his litt │ r an want

VALIDATION STATS:                                  loss: 1.26858,       accuracy: 0.61312

0%                                        Training on next 50 batches                                        100%
900000 (epoch 0) andersen_part10.txt │ ed itself round sometimes so q │ d tn  lf aound thme hmes ao su │ loss: 1.07577
900030 (epoch 0) andersen_part10.txt │ alled up through the tree, "No │ mled tp theough the woee  aIow │ loss: 0.95765
900060 (epoch 0) andersen_part10.txt │ se eyes were like towers. "Hel │ e nles were sike ah nrs  TTe d │ loss: 1.24676
900090 (epoch 0) andersen_part10.txt │  of this writhing would be.\\" │ wf thes waite ng warld be  \"I │ loss: 1.26019
900120 (epoch 0) andersen_part10.txt │  gilded all over. He was now a │ aovded anl tver  Te was aot a  │ loss: 1.14665
900150 (epoch 0) andersen_part10.txt │  It is living wicked men who d │ tt ws aikeng aitked aen aho wi │ loss: 1.42202
900180 (epoch 0) andersen_part10.txt │ at height that towns and villa │ t deaght 

VALIDATION STATS:                                  loss: 1.26322,       accuracy: 0.61556

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
rine!

"What we a way heavy and realing," said the little bed, "we will change my good old woman. It is neatle to cive here, and that is very sidishita; but it came to say in her heart. There was a very mourntrally time, but this was a very good sunk of it, he rome astend of were at home. The came of panes and grave-bushes, who was the sea things, and the country was like a boat and hearity on the feet countries were only our father's picture, for of evening the salants were fasting upon it. An this haup, standing from his own fatter and his sight, stood by the remoming of the first to love, and about the leaves and thinking; but they ware obliged to see him own statue, as if it was a give my old ping on convurisoment of it. "We shall we hane in her father whtn you cry, which you go n

...

VALIDATION STATS:                                  loss: 1.25002,       accuracy: 0.61972

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
are was the most beautiful ship that has confirmed. The shorts of her lofty maid has a great deal to him, but that is always a perform of a little cord with the storks of the world, the best thing that was. The star was always still in the water, and the child was a little chair, and the stork sat, and the soldier could not see them. The whole wife went to the stork's hand, and the children were always that the storks were to be a little child. The ships said that the shore was always to go on and therefore the stately country of the courtyard went to the sand-hills.

"I will not beat to tell me to the beautiful piece of man as a man and the woman who was a strange shore. I shall never be a greater prince where the state of the chimney shone in the wood, and then the countenance of th

VALIDATION STATS:                                  loss: 1.25497,       accuracy: 0.61752

0%                                        Training on next 50 batches                                        100%
5100000 (epoch 2) andersen_part12.txt │ s."\\"That may be," retorted t │  "\\"Ihet iay be " seported th │ loss: 0.89555
5100030 (epoch 2) andersen_part12.txt │ o should be for cutting it ope │ uwtorld be sougtot eng tn tuen │ loss: 1.44558
5100060 (epoch 2) andersen_part12.txt │ last his eyes closed, and he f │ ast tes fyes woosed  and te we │ loss: 0.76288
5100090 (epoch 2) andersen_part12.txt │ r like cobwebs.\\As John left  │ .oike ahnlars  \"n toan saft h │ loss: 1.83951
5100120 (epoch 2) andersen_part12.txt │ king, her father, was very muc │ ing  we  hather  whs sery luch │ loss: 0.89188
5100150 (epoch 2) andersen_part12.txt │ ed. Great black grasshoppers p │ d  "oeat Clack cress apper  wa │ loss: 1.34015
5100180 (epoch 2) andersen_part12.txt │ ttle with a few drops in it. H │ tl

VALIDATION STATS:                                  loss: 1.24925,       accuracy: 0.62027

0%                                        Training on next 50 batches                                        100%
5400000 (epoch 2) andersen_part17.txt │ he had an immense desire to ge │ e sad n  onmonse saecre wo tot │ loss: 1.15764
5400030 (epoch 2) andersen_part17.txt │  it. The world could get on wi │ tt  Ihe wirdd iould net an tht │ loss: 1.09465
5400060 (epoch 2) andersen_part17.txt │  to relate to him, and how man │ ah geaate to mem  and tew muny │ loss: 0.99548
5400090 (epoch 2) andersen_part17.txt │ in return was the three fern s │ n teaurn tit the soiee ooa  to │ loss: 1.52780
5400120 (epoch 2) andersen_part17.txt │ mantle was formed of thousands │ onyee was sirmed an thesgands  │ loss: 1.16882
5400150 (epoch 2) andersen_part17.txt │  then returned to the inn wher │ whe  seaurned to the cntaoiire │ loss: 1.16360
5400180 (epoch 2) andersen_part17.txt │ you possessed that he might re │ ou

VALIDATION STATS:                                  loss: 1.24856,       accuracy: 0.61959

┌───────────────────────────────────Generating random text from learned state───────────────────────────────────┐
ing in the world to be too much to be so made of the watchman, whose storks were so large that they were all only to be seen the water of a court beautiful man.

The General was a very said of the world that the boy was a good thing.

"I cannot see the word of the children of me and the storks of them was the stork of the world," said the old man, "that is all the windows on the house that had been built on the same things of the world, and the storks were the last, but the stone will be to be to be a bird who were to be seen in all this, they were not all to be to be the warm. The storks were to be a great child, and the wild beautiful maiden was not a little bird which had been buried into the water, and took the top of the story, as if they heard them, and she was about the window 

KeyboardInterrupt: 

# RNN's tale

In [6]:
ALPHASIZE = txt.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

# Data files can be downloaded from the following locations:
#    - Fully trained on Shakespeare or Tensorflow Python source:
#      https://drive.google.com/file/d/0B5njS_LX6IsDc2lWTmtyanRpOHc/view?usp=sharing
#    - Partially trained, to see how they make progress in training:
#      https://drive.google.com/file/d/0B5njS_LX6IsDUlFsMkdhclNSazA/view?usp=sharing

Andersen0 = "/Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_1519908456-0" # very first output  
Andersen1 = "/Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_1519939554-0"  # yesterday evening output
Andersen2 = "/Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_1519978793-3000000" # last training output

# use topn=10 for all but the last one which works with topn=2 for Andersen

author = Andersen2

ncnt = 0
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph(author + '.meta')
    new_saver.restore(sess, author)
    x = txt.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    file = open("generated_output.txt", "w")
    file.write("Hi there, this a generated output poem from the Andersen Machine. have fun! \n\n")
    for i in range(10000):
        yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})

        # If sampling is be done from the topn most likely characters, the generated text
        # is more credible and more "english". If topn is not set, it defaults to the full
        # distribution (ALPHASIZE)

        # Recommended: topn = 10 for intermediate checkpoints, topn=2 or 3 for fully trained checkpoints

        c = txt.sample_from_probabilities(yo, topn=2)
        y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(txt.convert_to_alphabet(c))
        print(c, end="")
        file.write(c)

        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            file.write("")
            ncnt = 0
    file.close() 

INFO:tensorflow:Restoring parameters from /Users/linda/Desktop/Bootcamp/Week7_Natural Language Processing/NLP project/checkpoints/rnn_train_1519978793-3000000
ite which we saw the street which were not a sidger of the stork of all things. So she came to her, 
and the large boy was so fine than the little girl was stronger that they were all the rest. The sto
rk was going in their heads and told the story of the window, and the little mouse stretched the str
eet and sang an one of the graves of the sun shone with their father.

"What are you arrived in the moonlight," said the ship said, "and then I have not been a story that 
must be able to go to the grandfather's. It is nearly a great deal of the world. I shall be my song 
about the willow, while I was a great deal of the same tome, and that we shall net be seen into a st
ranger and to look at a large countries of the world, and therefore she saw the most beautiful sight
. In a single day were all the little girl with the summer of t

e too large to the street. The stork were not the little boy who was a little child that shone with 
the summer which was a good country, as they were all the old companion who had been told the stork 
on this way, and took his feet as if to have been to see him. The little morned her for a single wat
er with his broad ship, which had been better to him, but the sun shone with the windows and stretch
ed out the stove, and said, "How made a good time that I must be a good morning, and I have never be
en to be seen and to love them again."

"I shall not be made a little, but I do not," said he, "that is not all this to be so dead."

The bottle was giving the beautiful sign of the stream which we could not be seen. The stranger was 
a grand forehead, which had been to be sure to have been a single world, and the lamp was so fine th
eir heads, as it was not a strength and the sun sat an old woman, whose lady was stiff and considere
d in the wall with her foot wite his branches, which were 

In [6]:
# cd /Users/linda/Desktop/Bootcamp/Week7_Natural\ Language\ Processing
# tensorboard --logdir log/
# output = link to produce loss and accuracy curves with TensorBoard